In [1]:
import pandas as pd

In [2]:
archivo = "Hackaton DB Final 04.21.xlsx"
xls = pd.ExcelFile(archivo)

In [3]:
dataframes = []

for sheet_name in xls.sheet_names:
    if sheet_name == "Boundary Conditions" or sheet_name == "Wafer Plan":
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    else: 
        df = pd.read_excel(xls, sheet_name=sheet_name)
    dataframes.append(df)

In [4]:
sheet_names = xls.sheet_names
sheet_names

['Supply_Demand',
 'Density per Wafer',
 'Weekly Demand Ratio',
 'Wafer Plan',
 'Boundary Conditions',
 'Yield',
 'Definitions',
 'Solution and Notes']

In [5]:
id_vars = dataframes[0].columns[:2]
value_vars = dataframes[0].columns[2:] 

dataframes[0] = dataframes[0].melt(id_vars=id_vars, value_vars=value_vars,
                   var_name='Period', value_name='Value')

dataframes[0]

,Product ID,Attribute,Period,Value
0,21A,Yielded Supply,Q2 95,0.000000e+00
1,21A,Safety Stock Target,Q2 95,0.000000e+00
2,21A,Safety Stock Target (WOS),Q2 95,0.000000e+00
3,21A,EffectiveDemand,Q2 95,0.000000e+00
4,21A,Total Projected Inventory Balance,Q2 95,1.413372e+08
...,...,...,...,...
625,23C,Safety Stock Target,Q4 03,NaN
626,23C,Safety Stock Target (WOS),Q4 03,1.420000e+00
627,23C,EffectiveDemand,Q4 03,3.669821e+09
628,23C,Total Projected Inventory Balance,Q4 03,-1.965593e+11


In [6]:
dataframes[4]

,0,1,2,3,4,5,6,7,8,9,...,435,436,437,438,439,440,441,442,443,444
0,NaN,NaN,Q3 95,Q3 95,Q3 95,Q3 95,Q3 95,Q3 95,Q3 95,Q3 95,...,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03,Q4 03
1,Product ID,Attribute,WW_09,WW_10,WW_11,WW_12,WW_13,WW_14,WW_15,WW_16,...,WW_26,WW_27,WW_28,WW_29,WW_30,WW_31,WW_32,WW_33,WW_34,WW_35
2,Total,Available Capacity,11037,10791,10265,10223,10181,11248,12023,12066,...,8697,8617,8070,8000,8000,8000,8000,8000,8000,8000
3,Total,Scheduled Capacity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total,Over/Under Capacity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21A,Available Capacity,5155,5258,5182,4840,4890,4830,4230,4373,...,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
6,21A,Scheduled Capacity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,21A,Over/Under Capacity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22B,Available Capacity,4000,4000,4000,4000,4000,4016,4316,4416,...,2500,2500,2500,2500,2500,2500,2500,2500,2500,2500
9,22B,Scheduled Capacity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_raw = dataframes[4]

quarters = df_raw.iloc[0]
weeks = df_raw.iloc[1]
combined_cols = [f"{q}_{w}" for q, w in zip(quarters, weeks)]

df = df_raw.copy()
df.columns = combined_cols

df = df.iloc[2:].reset_index(drop=True)

df.rename(columns={df.columns[0]: 'Product ID', df.columns[1]: 'Attribute'}, inplace=True)

df_melted = df.melt(id_vars=['Product ID', 'Attribute'],
                    var_name='Period',
                    value_name='Value')

df_melted[['Quarter', 'Week']] = df_melted['Period'].str.split('_', n=1, expand=True)
df_melted.drop(columns='Period', inplace=True)

dataframes[4] = df_melted
dataframes[4]

,Product ID,Attribute,Value,Quarter,Week
0,Total,Available Capacity,11037,Q3 95,WW_09
1,Total,Scheduled Capacity,NaN,Q3 95,WW_09
2,Total,Over/Under Capacity,NaN,Q3 95,WW_09
3,21A,Available Capacity,5155,Q3 95,WW_09
4,21A,Scheduled Capacity,NaN,Q3 95,WW_09
...,...,...,...,...,...
5311,22B,Scheduled Capacity,NaN,Q4 03,WW_35
5312,22B,Over/Under Capacity,NaN,Q4 03,WW_35
5313,23C,Available Capacity,500,Q4 03,WW_35
5314,23C,Scheduled Capacity,NaN,Q4 03,WW_35


In [8]:
df_raw = dataframes[3]

quarters = df_raw.iloc[0]
weeks = df_raw.iloc[1]
combined_cols = [f"{q}_{w}" for q, w in zip(quarters, weeks)]

df = df_raw.copy()
df.columns = combined_cols

df = df.iloc[2:].reset_index(drop=True)

df.rename(columns={df.columns[0]: 'Design Handle'}, inplace=True)

df_melted = df.melt(id_vars=['Design Handle'],
                    var_name='Period',
                    value_name='Value')

df_melted['Value'] = 0

df_melted[['Quarter', 'Week']] = df_melted['Period'].str.split('_', n=1, expand=True)
df_melted.drop(columns='Period', inplace=True)

dataframes[3] = df_melted
dataframes[3]

,Design Handle,Value,Quarter,Week
0,Total,0,Q3 95,WW_09
1,21A,0,Q3 95,WW_09
2,22B,0,Q3 95,WW_09
3,23C,0,Q3 95,WW_09
4,Total,0,Q3 95,WW_10
...,...,...,...,...
1767,23C,0,Q4 03,WW_34
1768,Total,0,Q4 03,WW_35
1769,21A,0,Q4 03,WW_35
1770,22B,0,Q4 03,WW_35


In [9]:
uniquePeriods = dataframes[0]['Period'].unique()

periods = pd.DataFrame(uniquePeriods, columns=['Period'])
periods['Index'] = range(1, len(periods) + 1)

In [10]:
with pd.ExcelWriter('DB.xlsx') as writer:
    for i, name in enumerate(sheet_names):
        dataframes[i].to_excel(writer, sheet_name=name, index=False)
    periods.to_excel(writer, sheet_name='Periods', index=False)
